<a href="https://colab.research.google.com/github/Patojukarthikeya73/Infosys_Springboard/blob/main/Module2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1 — Install dependencies
!pip install --quiet transformers gradio accelerate sentencepiece

# Step 1 — Imports and setup
import ast
import re
import gradio as gr
from typing import Optional

MODEL_CACHE = {}   # cache for loaded models


In [2]:
# Step 2 — Define AST-based Python explanation function
def explain_python_ast(code: str) -> str:
    """
    Produce a readable explanation of Python code using AST parsing.
    Explains top-level constructs (functions, classes, imports, loops, etc.)
    """
    try:
        tree = ast.parse(code)
    except Exception as e:
        return f"❌ Could not parse code as Python AST: {e}"

    explanations = []

    def explain_args(args):
        return ", ".join([arg.arg for arg in args.args])

    for node in tree.body:
        if isinstance(node, ast.FunctionDef):
            name = node.name
            args = explain_args(node.args)
            doc = ast.get_docstring(node) or ""
            explanations.append(f"- Function `{name}({args})` defined. {('Doc: '+doc) if doc else ''}")

            for sub in ast.walk(node):
                if isinstance(sub, ast.For):
                    explanations.append(f"  * Inside `{name}`: for-loop iterates over {getattr(ast, 'unparse', lambda x:'iterable')(sub.iter)}.")
                if isinstance(sub, ast.While):
                    explanations.append(f"  * Inside `{name}`: while-loop with condition {getattr(ast, 'unparse', lambda x:'condition')(sub.test)}.")
                if isinstance(sub, ast.If):
                    explanations.append(f"  * Inside `{name}`: if-statement checks {getattr(ast, 'unparse', lambda x:'condition')(sub.test)}.")

        elif isinstance(node, ast.ClassDef):
            explanations.append(f"- Class `{node.name}` defined.")
        elif isinstance(node, ast.Import):
            names = ", ".join([alias.name for alias in node.names])
            explanations.append(f"- Imported module(s): {names}.")
        elif isinstance(node, ast.ImportFrom):
            mod = node.module or ""
            names = ", ".join([alias.name for alias in node.names])
            explanations.append(f"- From `{mod}` import: {names}.")
        elif isinstance(node, ast.Assign):
            targets = ", ".join([ast.unparse(t) for t in node.targets]) if hasattr(ast, "unparse") else "variable"
            explanations.append(f"- Assignment to `{targets}`.")
        elif isinstance(node, ast.Expr):
            if isinstance(node.value, ast.Constant) and isinstance(node.value.value, str):
                explanations.append(f"- Docstring/comment: {node.value.value[:80]!r}...")
        else:
            explanations.append(f"- Top-level {node.__class__.__name__} encountered.")

    return "Python AST Explanation:\n" + "\n".join(explanations)


In [3]:
# Step 3 — Lazy model loader
def get_generator(model_key: str):
    from transformers import pipeline
    import torch

    device = 0 if torch.cuda.is_available() else -1

    if model_key in MODEL_CACHE:
        return MODEL_CACHE[model_key]

    if model_key == "phi":
        model_name = "microsoft/Phi-3-mini-4k-instruct"
    elif model_key == "codegen":
        model_name = "Salesforce/codegen-350M-mono"
    else:
        raise ValueError("Unknown model key")

    pipe = pipeline("text-generation", model=model_name, tokenizer=model_name, device=device, max_length=512)
    MODEL_CACHE[model_key] = pipe
    return pipe


In [4]:
# Step 4 — Generation + Explanation logic
def model_generate_text(lang: str, mode: str, user_input: str) -> str:
    lang, mode = lang.lower(), mode.lower()
    model_key = "phi" if lang == "python" else "codegen"
    gen = get_generator(model_key)

    if mode == "generate":
        prompt = f"Write only {lang} code for the following instruction:\n{user_input}\n\n### {lang} code:\n"
    else:
        prompt = f"Explain the following {lang} code clearly and briefly:\n{user_input}\n\nExplanation:\n"

    try:
        out = gen(prompt, max_new_tokens=256, do_sample=False, temperature=0.2)
        text = out[0]["generated_text"].replace(prompt, "").strip()
        text = re.sub(r"```(?:\w+)?", "", text)
        return text
    except Exception as e:
        return f"❌ Model generation error: {e}"


In [5]:
# Step 5 — Gradio UI setup
with gr.Blocks(title="CodeGenie — Module 2") as demo:
    gr.Markdown("# 🧠 CodeGenie: Generate & Explain Code")
    with gr.Row():
        with gr.Column():
            lang = gr.Dropdown(["Python", "JavaScript", "Solidity"], label="Language", value="Python")
            mode = gr.Radio(["Generate", "Explain"], label="Mode", value="Generate")
            use_ast = gr.Checkbox(label="Use AST for Python explanation", value=True)
            user_input = gr.Textbox(label="Prompt / Code", lines=8, placeholder="Enter prompt or paste code")
            run_btn = gr.Button("Run 🚀")
        with gr.Column():
            output = gr.Textbox(label="Output (Code or Explanation)", lines=18)

    def handle_run(selected_lang, selected_mode, ast_flag, user_text):
        if not user_text.strip():
            return "⚠️ Please enter some input."
        if selected_mode.lower() == "explain" and selected_lang.lower() == "python" and ast_flag:
            ast_part = explain_python_ast(user_text)
            model_part = model_generate_text("python", "explain", user_text)
            return f"{ast_part}\n\n---\n\nModel-based Explanation:\n{model_part}"
        return model_generate_text(selected_lang, selected_mode, user_text)

    run_btn.click(handle_run, [lang, mode, use_ast, user_input], [output])

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://10c0599ec0e1a053bb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
